# Task 3 Traffic Sign Classification / Recognition System

## Loading files

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
df = pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')
df_test = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

In [ ]:
df.head()

In [ ]:
#Contains Path to each image directory
#Class id is the target variable

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
#Path is type of object , need to convert it to string for accessing images through it.

In [ ]:
df_test.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
meta = pd.read_csv('../input/gtsrb-german-traffic-sign/Meta.csv')

In [ ]:
meta.head()

## Building model

In [ ]:
import keras 
from matplotlib import pyplot as plt
from keras.applications import Xception

In [ ]:
model = keras.models.Sequential([
    Xception(include_top=False, weights=None, input_shape=(299, 299, 3), pooling='avg'),
    keras.layers.Dense(43),
    keras.layers.Softmax()]
    )
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# model = Sequential()
# model.add(Xception(include_top=False, weights=None, input_shape=(299, 299, 3), pooling='avg'))
# model.add(Dense(43))
# model.add(Softmax())
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy']
# )
model.summary()

## Preparing data for model training

In [ ]:
df['ClassId'] = df['ClassId'].astype(str)
df_test['ClassId'] = df_test['ClassId'].astype(str)
for i in range(0, len(df['ClassId'])):
    if len(df['ClassId'][i]) == 1:
        df['ClassId'][i] = '0' + df['ClassId'][i]
for i in range(0, len(df_test['ClassId'])):
    if len(df_test['ClassId'][i]) == 1:
        df_test['ClassId'][i] = '0' + df['ClassId'][i]

print(df['ClassId'][20], df['Path'][20])
img = Image.open('../input/gtsrb-german-traffic-sign/' + df['Path'][20])
plt.imshow(img)

In [ ]:
gen_train = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,shear_range=0.2)
gen_test = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
generator_train = gen_train.flow_from_dataframe(dataframe=df,
                                                directory='../input/gtsrb-german-traffic-sign/',
                                                target_size=(299, 299),
                                                x_col='Path',
                                                y_col='ClassId',
                                                batch_size=32,
                                                class_mode='categorical'
                                                )

In [ ]:
generator_test = gen_test.flow_from_dataframe(dataframe=df_test,
                                            directory='../input/gtsrb-german-traffic-sign/',
                                            x_col='Path',
                                            y_col='ClassId',
                                            target_size=(299, 299),
                                            batch_size=16,
                                            class_mode='categorical'
                                            )

## Training model

In [ ]:
history = model.fit(generator_train, epochs=2, verbose=1)

In [ ]:
model.evaluate(generator_test, verbose=1)

In [ ]:
batch = 2
plt.figure(figsize=(64, 64))
plt.tight_layout()
for i in range(0, 16):
    plt.subplot(16, 2, 2 * i + 1), plt.axis('off')
    plt.imshow(generator_test[batch][0][i])
    pre = model.predict_classes(generator_test[batch][0])
    img = Image.open('../input/gtsrb-german-traffic-sign/Meta/' + str(pre[i]) + '.png')
    plt.subplot(16, 2, 2 * i + 2), plt.axis('off')
    plt.imshow(img)